# QUICKSTART
기본적인 기계 학습을 구현하기 위한 API 사용에 대해 다룸

---
# 1. Working with data
- 파이토치에서 데이터를 다룰 때 핵심이 되는 두 가지
1. **torch.utils.data.Dataset** : sample과 그에 맞는 label을 저장
2. **torch.utils.data.DataLoader** : Dataset을 iterable 하게 wrap

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

- 파이토치는 **TorchText, TorchVision, TorchAudio**와 같은 domain-specific 라이브러리를 제공함
- 각 라이브러리엔 데이터셋이 포함됨
- 여기선 TorchVision 의 데이터셋을 사용해볼 것

> **torchvision.datasets**
>- CIFAR, COCO 와 같은 real-wordl vision data 인 Dataset 객체를 포함함
>- 여기선 **FashionMNIST** 데이터셋을 사용해볼 것
>- 모든 TorchVisio 의 Dataset은 두 가지 arguements를 포함함: **transform, target_transform**
    >- 이는 sample과 그 label을 조작하기 위함임

In [3]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

- 다운받은 Dataset을 DataLoader의 인자로 넣을 것
- DataLoader는 Dataset을 감싸 **automatic batching, sampling, shiffling, multiprocess data loading** 등을 도와줌
- 여기선 batch_size 를 64로 정의함 : dataloader의 각 요소는 64개의 feature와 label를 반환

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.int64


---
# 2. Creating Models
- 파이토치에서 인공신경망을 정의하기 위해선 **nn.Module** 을 상속한 클래스를 만들어야 함
- **init(생성자 메서드)** 에서 네트워크를 구성하는 layer들을 정의
- **forward** 메서드에서 네트워크가 데이터를 어떻게 처리하는지 정의
- 인공신경망의 연산을 가속하기 위해 네트워크를 **GPU** 에 옮겨볼 것

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


---
# 3. Optimizing the Model Parameters
- 모델을 학습하기 위해선 **loss function 과 optimizer** 가 필요함

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

>**학습 루프에서 일어나는 작업**
>1. 모델이 학습 데이터셋에 대해 예측 (배치로 입력받음)
>2. 예측한 결과와 정답값을 비교하여 loss를 계산
>3. loss를 줄이는 방향으로 모델의 파라미터를 수정하기 위해 loss를 backpropagate

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

- 학습이 잘 되고 있는지 검증하기 위해 테스트셋에 대한 성능도 확인해야 함

In [11]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- 학습은 학습 데이터셋 전체에 대한 몇 번의 반복으로 진행됨.
- 각 반복을 **epoch** 이라고 함
- 각 epoch 동안 모델은 더 나은 예측을 위해 파라미터를 학습함
- 매 epoch마다 정확도는 올라가고, 로스는 내려가는 것이 이상적

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296429 [    0/60000]
loss: 2.291669 [ 6400/60000]
loss: 2.279853 [12800/60000]
loss: 2.286928 [19200/60000]
loss: 2.262813 [25600/60000]
loss: 2.235382 [32000/60000]
loss: 2.256296 [38400/60000]
loss: 2.220906 [44800/60000]
loss: 2.221251 [51200/60000]
loss: 2.216681 [57600/60000]
Test Error: 
 Accuracy: 46.2%, Avg loss: 0.034812 

Epoch 2
-------------------------------
loss: 2.197034 [    0/60000]
loss: 2.208733 [ 6400/60000]
loss: 2.181816 [12800/60000]
loss: 2.226074 [19200/60000]
loss: 2.163205 [25600/60000]
loss: 2.097574 [32000/60000]
loss: 2.162460 [38400/60000]
loss: 2.086795 [44800/60000]
loss: 2.096765 [51200/60000]
loss: 2.091247 [57600/60000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 0.032961 

Epoch 3
-------------------------------
loss: 2.059196 [    0/60000]
loss: 2.081915 [ 6400/60000]
loss: 2.030728 [12800/60000]
loss: 2.132223 [19200/60000]
loss: 1.996475 [25600/60000]
loss: 1.890053 [32000/60000]
loss: 2.007692 [38400/

---
# 4. Saving Models
- 모델을 저장하기 위한 방법으로 주로 모델을 **state dictionary로 serialize 함**

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


---
# 5. Loading Models
>**모델을 로드하는 방법**
>1. 초기 모델 구조를 생성
>2. 저장한 파라미터를 로드

In [15]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

- 로드한 모델로 예측하기

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
